<h1><center><span style="font-family:Comic Sans MS">Scraping Data About Android Phones On Jumia</span><center><h1>

<strong><span style="font-family:Lucida Sans">The aim of this project is to scrape all models of Android phones hosted for sale on the Jumia website. The data gotten from here would be saved in a "csv" format for easy analysis later</span><strong>

In [1]:
#Importing packages

from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import numpy as np
from time import sleep
from time import time


In [2]:
#First we create sets of empty list to load our data into later 

description =[]
current_prices = []
ratings = []
old_prices = []
brands = []
discounts = []
models = []
stars =[]

# time() is the currnet time at any point of running the code
start_time = time()

# initial request is set to zero and increased by 1 on every successful request
request = 0

#Create a loop to iterate through the pages we would be scraping, in this case 50 pages
for i in range (1,51):
    
    link = "https://www.jumia.com.ng/android-phones/?operating_system=Android&page={}#catalog-listing".format(i)
    page = requests.get(link)
    request += 1
#  displays each request so we that can monitor the process,not very necessary,you can comment it out.
    #print('request: {}'.format(request))
    
#We parse each request through BeautifulSoup 
    pages = soup(page.text , "html.parser")
    
#We find the class that contains all the information we want to scrape after inspecting the webpage
    containers = pages.find_all('article', class_="prd _fb col c-prd")
#We iterate through every item we want to get and then add it to the empty list we created earlier.
    for container in containers:
        
        #Brands
        brand = container.find('h3', class_='name').text.split(' ')[0]
        brands.append(brand)
        
        #Phone models
        phone_model = container.find('h3', class_='name').text.split(' ')[:3]
        model= ' '.join(phone_model)
        models.append(model)
        
        #Phone Descriptions
        descrip = container.find('h3', class_='name').text.split(' ')[:]
        descriptions= ' '.join(descrip)
        description.append(descriptions)
        
        #Current Prices
        cprice = container.find('div', class_='prc').text.split(' ')[:]
        current_price= ' '.join(cprice)
        current_prices.append(current_price)

#Some products have some value applicable e.g Discount, hence we create an exception that would append "None" if no value was found
        
        #Old Prices
        try:
            oldprice= container.find('div', class_='old').text.split(' ')[:]
            old_price= ' '.join(oldprice)
            old_prices.append(old_price)
        except Exception:
            oldprice = None
            old_prices.append(oldprice)
            
        #Discount
        try:
            dscount = container.find('div', class_='tag _dsct _sm').text.split(' ')[:]
            discount= ' '.join(dscount)
            discounts.append(discount)
        except Exception:
            dscount = None
            discounts.append(dscount)
        
        #Ratings
        try:
            rate = container.find('div', class_='rev').text.replace("(","").replace(")","").replace("5"," ").replace(" "," ")[10:]
            rating= rate.strip()
            ratings.append(rating)
        except Exception:
            rate = None
            ratings.append(rate)
            
        #Number of Stars
        try:

            no_star = container.find('div', class_='rev').text.split(' ')[:1]
            star= ' '.join(no_star)
            stars.append(star)
        except Exception:
            no_star = None
            stars.append(no_star)
            
#We then create a DataFrame to store all the items and values.
Android_phones = pd.DataFrame({'Brand': brands, 'Phone Model': models,'Old Price': old_prices,'Discounts': discounts,
                                 'description': description,'Ratings': ratings,'Stars': stars, 'Current Prices':current_prices}, columns =[
        'Brand','Phone Model','Current Prices','Old Price','Discounts','description','Ratings','Stars'])
elasped_time = time() - start_time
print("Completed in {}secs".format(elasped_time))

Completed in 82.31272745132446secs


<span style="font-family:Lucida Sans">We can do a quick view of the Data</span>

In [3]:
#We were able to scrape a total of 2000 items
Android_phones.shape

(2000, 8)

In [4]:
#A quick view of the first 10 rows
Android_phones.head(10)


,Brand,Phone Model,Current Prices,Old Price,Discounts,description,Ratings,Stars
0,Gionee,Gionee S11 Lite,"₦ 39,990","₦ 49,990",20%,"Gionee S11 Lite 5.7-Inch HD (4GB,64GB ROM) And...",2368,4.1
1,Lava,Lava Z50 PRO,"₦ 14,000","₦ 19,000",26%,"Lava Z50 PRO - 5.0"" -1GB RAM 8GB ROM - 5MP Rea...",139,4.2
2,UMIDIGI,UMIDIGI A7S Infrared,"₦ 33,400","₦ 63,690",48%,UMIDIGI A7S Infrared Temperature Sensor 6.53-I...,17,4.1
3,UMIDIGI,UMIDIGI A7S Infrared,"₦ 33,400","₦ 63,690",48%,UMIDIGI A7S Infrared Temperature Sensor 6.53-I...,74,4.1
4,Samsung,Samsung Galaxy A51,"₦ 118,430","₦ 140,000",15%,"Samsung Galaxy A51 6.5'' (6GB,128GB ROM) Andro...",146,4.5
5,UMIDIGI,UMIDIGI BISON IP68/IP69K,"₦ 72,900","₦ 120,900",40%,"UMIDIGI BISON IP68/IP69K (6GB,128GB) 6.3-Inch ...",22,4.7
6,Cubot,"Cubot Note 7,","₦ 31,200","₦ 41,099",24%,"Cubot Note 7, 5.5 Inches,4G LTE,2GB + 16GB,310...",90,3.8
7,UMIDIGI,UMIDIGI A9 Pro,"₦ 69,500","₦ 112,600",38%,UMIDIGI A9 Pro Infrared Temperature Sensor (48...,20,4.8
8,LOGY,LOGY 6.6 Inch,"₦ 31,300","₦ 98,796",68%,LOGY 6.6 Inch HD Water Drop Screen Face Recogn...,None,None
9,Samsung,Samsung Galaxy A12,"₦ 70,990","₦ 79,900",11%,"Samsung Galaxy A12 - 6.5"" 48MP Camera, 4/64GB ...",4,4.8


In [5]:
#A quick view of the last 10 rows
Android_phones.tail(10)


,Brand,Phone Model,Current Prices,Old Price,Discounts,description,Ratings,Stars
1990,Meeysoo,Meeysoo P45 Pro,"₦ 29,990","₦ 87,780",66%,Meeysoo P45 Pro Face ID 6.6 Inch （128GB Free ...,None,None
1991,yeamuds,yeamuds 6.6〞HD Smartphone,"₦ 49,786","₦ 63,380",21%,yeamuds 6.6〞HD Smartphone Deca Core 64GB ROM +...,None,None
1992,Meeysoo,Meeysoo Global Version,"₦ 36,680","₦ 146,390",75%,Meeysoo Global Version R4 Android10.0 Mobile P...,None,None
1993,XIAOMI,XIAOMI Redmi Note,"₦ 124,150","₦ 177,350",30%,"XIAOMI Redmi Note 9S, 48MP Camera, 6GB+128GB, ...",None,None
1994,Samsung,"Samsung Galaxy A12,","₦ 119,999","₦ 143,999",17%,"Samsung Galaxy A12, 6.5-inches (4GB RAM, 64GB ...",None,None
1995,Genera,Genera 512MB+4GB Android,"₦ 19,044",None,None,Genera 512MB+4GB Android Smartphone5.8 Inch Sc...,None,None
1996,Meeysoo,Meeysoo N30U 6.9-Inch,"₦ 49,500","₦ 87,780",44%,"Meeysoo N30U 6.9-Inch (3GB RAM, 32GB ROM+128GB...",None,None
1997,Tecno,Tecno POP2 Plus,"₦ 46,500","₦ 63,200",26%,Tecno POP2 Plus (BA2) Android SmartPhone - 6 I...,None,None
1998,Vivo,"Vivo Y91C, 6.22-Inch,","₦ 52,000",None,None,"Vivo Y91C, 6.22-Inch, 2GB RAM 32GB ROM Android...",None,None
1999,Meeysoo,Meeysoo Mate 45,"₦ 33,500","₦ 87,780",62%,Meeysoo Mate 45 Pro 3G Smartphone Android 9.0 ...,None,None


In [6]:
Android_phones.describe()

,Brand,Phone Model,Current Prices,Old Price,Discounts,description,Ratings,Stars
count,2000,2000,2000,1428,1428,2000,350,350
unique,87,668,803,443,77,1485,54,24
top,Meeysoo,Samsung Galaxy Note,"₦ 33,500","₦ 87,780",62%,Genera 512MB+4GB Android Smartphone5.0-5.8 Inc...,1,5
freq,407,76,115,202,116,8,141,87


In [7]:
Android_phones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Brand           2000 non-null   object
 1   Phone Model     2000 non-null   object
 2   Current Prices  2000 non-null   object
 3   Old Price       1428 non-null   object
 4   Discounts       1428 non-null   object
 5   description     2000 non-null   object
 6   Ratings         350 non-null    object
 7   Stars           350 non-null    object
dtypes: object(8)
memory usage: 125.1+ KB


In [9]:
#Saving the data as a CSV file
Android_phones.to_csv("Android_Phones.csv", index =False)

<br>

<h2><center><span style="font-family:Comic Sans MS">Thank You For Viewing This Notebook</span><center><h2>